In [1]:
import numpy as np
import nibabel as nib

import os
import shutil

from utils import read_tracking, write_tracking

In [2]:
config = {
    "spine": {
        "images": "C:/Users/Bugra/Desktop/masters-thesis/spine_all_sweeps.nii.gz",
        "ultranerf": "C:/Users/Bugra/Desktop/masters-thesis/spine_all_sweeps_conf.nii.gz",
        "tracking": "C:/Users/Bugra/Desktop/masters-thesis/spine_all_sweeps_tracking.csv",
    },
    "liver": {
        "images": "C:/Users/Bugra/Desktop/masters-thesis/liver_all_sweeps.nii.gz",
        "ultranerf": "C:/Users/Bugra/Desktop/masters-thesis/liver_all_sweeps_conf.nii.gz",
        "tracking": "C:/Users/Bugra/Desktop/masters-thesis/liver_all_sweeps_tracking.csv",
    },
    "leg": {
        "images": "C:/Users/Bugra/Desktop/masters-thesis/leg_data.nii.gz",
        "ultranerf": "C:/Users/Bugra/Desktop/masters-thesis/leg_conf.nii.gz",
        "tracking": "C:/Users/Bugra/Desktop/masters-thesis/leg_tracking.csv",
    },
}

In [3]:
splits = {
    "spine": [0, 159, 313, 465, 618, 769, 921, 1067, 1217, 1370, 1534, 1682, 1823, 1975, 2123, 2253, 2389],
    "liver": [0, 200, 400, 600, 800, 1000, 1200],
    "leg": [0, 361, 822, 1151, 1441, 1714],
}

In [4]:
dest_dir = "07032024"
os.makedirs(dest_dir, exist_ok=True)

In [5]:
for key, value in config.items():
    images = nib.load(value["images"])
    ultranerf = nib.load(value["ultranerf"])
    tracking = read_tracking(value["tracking"])

    print(f"Processing {key}")
    print(f"{images.shape=}")
    print(f"{ultranerf.shape=}")
    print(f"{tracking.shape=}")

    key_dir = os.path.join(dest_dir, key)
    if os.path.exists(key_dir):
        shutil.rmtree(key_dir)
    os.makedirs(key_dir)

    for i in range(len(splits[key]) - 1):
        start = splits[key][i]
        end = splits[key][i + 1]
        print(start, end)

        images_split = images.get_fdata()[:, :, start:end]
        ultranerf_split = ultranerf.get_fdata()[:, :, start:end]
        tracking_split = tracking[start:end]

        images_split = images_split.astype(np.uint8)
        ultranerf_split = (ultranerf_split * 255).astype(np.uint8)

        images_split = nib.Nifti1Image(images_split, images.affine)
        ultranerf_split = nib.Nifti1Image(ultranerf_split, ultranerf.affine)

        nib.save(images_split, os.path.join(key_dir, f"{i}_images.nii.gz"))
        nib.save(ultranerf_split, os.path.join(key_dir, f"{i}_ultranerf.nii.gz"))
        write_tracking(os.path.join(key_dir, f"{i}_tracking.csv"), tracking_split)

Processing spine
images.shape=(164, 1300, 2389)
ultranerf.shape=(164, 1300, 2389)
tracking.shape=(2389, 4, 4)
0 159
159 313
313 465
465 618
618 769
769 921
921 1067
1067 1217
1217 1370
1370 1534
1534 1682
1682 1823
1823 1975
1975 2123
2123 2253
2253 2389
Processing liver
images.shape=(256, 516, 1200)
ultranerf.shape=(256, 516, 1200)
tracking.shape=(1200, 4, 4)
0 200
200 400
400 600
600 800
800 1000
1000 1200
Processing leg
images.shape=(221, 537, 1714)
ultranerf.shape=(221, 537, 1501)
tracking.shape=(1714, 4, 4)
0 361
361 822
822 1151
1151 1441
1441 1714
